# Exercise in Click Log Analysis

**Goal**: Identify which combinations of `Server` and `Domain` have issues with clicks that take a long time (i.e. `Big Clicks`).

In [15]:
%run supporting_functions.py
set_sns()
df, descriptives = load_data();
clicktime = df['Click Time (+)'].dropna()
rendtime = df['Render Time (+)'].dropna()
percent_cutoff = 95
ax = plot_hists(clicktime, percentile=percent_cutoff, normed=False, describe_inset=False, xlabel='Click Time')
ax.spines['right'].set_visible(False); 
ax.spines['top'].set_visible(False); 
ax.figure.tight_layout();
save_plot(figh=ax.figure, outname='plot_clicktime_upper5omitted.png');
df2md(descriptives, ncol2bold=1);


FIGURE SAVED TO: plot_clicktime_upper5omitted.png
|  index  |Click Time|Click Time (+)|Render Time|Render Time (+)|
|---------|---------:|-------------:|----------:|--------------:|
|**count**|   44741.0|       17602.0|    44741.0|        16207.0|
|**mean** |    5214.5|       13255.7|    -1401.1|          108.7|
|**std**  |  177407.0|      282657.6|   296696.6|          183.4|
|**min**  |      -1.0|           1.0|-62734870.0|            2.0|
|**25%**  |      -1.0|         511.0|       -1.0|           21.0|
|**50%**  |       0.0|         764.0|       -1.0|           56.0|
|**75%**  |     603.0|        1664.7|       27.0|          134.0|
|**max**  |15553165.0|    15553165.0|     4143.0|         4143.0|



<Figure size 432x288 with 0 Axes>

# Dataset Overview

- most of the 44,741 records contained invalid (non-positive) values for both click (61%) and render (64%) times 
    + these times are excluded from the primary analysis (see final section for a closer look at this invalid times)
- the table shows the *Mean*, *Standard Deviation (SD)*, *Minimum*,  *25th*, *50th*, and *75th* percentiles, and the *Maximum* for the click and render times both before (raw) and after (+) excluding non-positive values


|   |Click Time (raw) |Click Time (+)|Render Time (raw)|Render Time (+)|
|---------|---------:|-------------:|----------:|--------------:|
|**N**|   44741 |       17602 |    44741 |        16207 |
|**Mean** |    52145|       132557|    -14011|          1087|
|**SD**  |  177407 |      2826576|   2966966|          1834|
|**Min**  |      -1 |           1 |-62734870 |            2 |
|**25%**  |      -1 |         511 |       -1 |           21 |
|**50%**  |       0 |         764 |       -1 |           56 |
|**75%**  |     603 |        16647|       27 |          134 |
|**Max**  |15553165 |    15553165 |     4143 |         4143 |

## Click time (+) distribution

![](/plot_clicktime_upper5omitted.png)

In [3]:
rho = df[['Click Time (+)', 'Render Time (+)']].corr(method='spearman')
NUM_BINS = 12
figh, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6));
sns.regplot(df['Render Time (+)'], df['Click Time (+)'], ax=ax1, fit_reg=False);
ax1.set_title('rank correlation = {:.2f}'.format(rho.iloc[0,-1]));
ax2.set_ylabel('Click Time'); ax2.set_xlabel('Render Time');
sns.regplot(df['Render Time (+)'], df['Click Time (+)'], ax=ax2, x_bins=NUM_BINS, fit_reg=False);
ax2.set_title('{:d} equally-sized bins'.format(NUM_BINS));
ax2.set_ylabel('Mean Click Time'); ax2.set_xlabel('Render Time (Bin Mininum)');
format_yticks(ax1); format_yticks(ax2); format_xticks(ax1);
figh.tight_layout();
save_plot(figh=figh, outname='plot_clickxrender.png');


FIGURE SAVED TO: plot_clickxrender.png


## The relationship between Click time and Render time

- Left pane is a scatter plot of the raw, positive-valued Click and Render times, which showed a moderately positive rank-order correlation
- Right pane plots the average Click times for 12 equally-sized Render time bins to better visualize this positive correlation

![](/plot_clickxrender.png)

## Combinations of Server and Domain that have the biggest issues with clicks that take a long time (i.e. Big Clicks)

- table shows *Mean*, *Standard Deviation (SD)*, and *Maximum* for the 10 Server-Domain combos with the highest mean click times
- the top 4 combos are distinguished by time distributions with big means and big variances

|        Server        |    Domain     |*N* |Mean  |  SD  |  Max   |
|----------------------|---------------|----:|----:|-----:|-------:|
|**CoreProcesses**     |**HCM**        |  578|39113|460223| 8830183|
|**FunctionalSetup**   |**Common**     | 4605|30598|468640|15553165|
|**CRMCommon**         |**CRM**        |  320|23773|401079| 7175832|
|**Payable**           |**Financial**  | 1401|17517|288432| 9326914|
|**HelpPortal**        |**Common**     |   24|10977| 41673|  205250|
|**SCMCommon**         |**SCM**        |   45| 4189| 13660|   71375|
|**HomePage**          |**Common**     | 8709| 3792|108275| 9095226|
|**Procurement**       |**Procurement**|   46| 3413| 11667|   79476|
|**ContractManagement**|**CRM**        |    7| 3214|  3812|   11173|
|**GeneralLedger**     |**Financial**  |  284| 2270|  7188|  116897|

In [18]:
# df.dropna(subset=['Click Time (+)'], inplace=True)
big_click_cutoffs = [4000, 8000, 12000]
name = big_click_cutoffs.copy()
for idx,c in enumerate(big_click_cutoffs):
    name[idx] = '{:d}+ Clicks'.format(c)
    df[name[idx]] = df['Click Time (+)'] >= c

In [19]:
pt = df.pivot_table(values='Click Time (+)', index=['Server', 'Domain'], dropna=True, margins=False, aggfunc=['count', 'mean', 'median', 'std', 'max'])
pt.columns = ['Count', 'Mean', 'Median', 'SD', 'Max']
pt = pt.sort_values(by='Mean', ascending=False).apply(lambda x: x.astype(int)).head(10)
df2md(pt, ncol2bold=2)

|        Server        |    Domain     |Count|Mean |Median|  SD  |  Max   |
|----------------------|---------------|----:|----:|-----:|-----:|-------:|
|**CoreProcesses**     |**HCM**        |  578|39113|  1045|460223| 8830183|
|**FunctionalSetup**   |**Common**     | 4605|30598|   993|468640|15553165|
|**CRMCommon**         |**CRM**        |  320|23773|   574|401079| 7175832|
|**Payable**           |**Financial**  | 1401|17517|   931|288432| 9326914|
|**HelpPortal**        |**Common**     |   24|10977|   517| 41673|  205250|
|**SCMCommon**         |**SCM**        |   45| 4189|   984| 13660|   71375|
|**HomePage**          |**Common**     | 8709| 3792|   539|108275| 9095226|
|**Procurement**       |**Procurement**|   46| 3413|   951| 11667|   79476|
|**ContractManagement**|**CRM**        |    7| 3214|  1564|  3812|   11173|
|**GeneralLedger**     |**Financial**  |  284| 2270|  1193|  7188|  116897|



In [24]:
g = df.dropna().groupby(['Server', 'Domain'])[name]
p = 100*(g.sum() / g.count()).dropna().sort_values(by=name[0], ascending=False).head(10)
df2md(p, ncol2bold=2)

|        Server        |    Domain     |4000+ Clicks|8000+ Clicks|12000+ Clicks|
|----------------------|---------------|-----------:|-----------:|------------:|
|**ContractManagement**|**CRM**        |       28.57|       14.29|         0.00|
|**CoreProcesses**     |**HCM**        |       11.46|        7.05|         4.59|
|**Procurement**       |**Procurement**|       10.87|        6.52|         2.17|
|**SCMCommon**         |**SCM**        |        8.89|        6.67|         4.44|
|**FunctionalSetup**   |**Common**     |        7.93|        2.85|         1.71|
|**Payable**           |**Financial**  |        7.42|        2.33|         1.02|
|**FinancialCommon**   |**Financial**  |        7.06|        3.45|         2.04|
|**GeneralLedger**     |**Financial**  |        7.01|        1.11|         1.11|
|**CoreSetup**         |**HCM**        |        6.81|        3.19|         2.23|
|**HelpPortal**        |**Common**     |        5.00|        5.00|         5.00|



# Other useful insights

## Non-Positive Click Times: evenly distributed across Server-Domains combos?

- counts of _Negative_, _Positive_, and _Zero_ values observed for the 10 Server-Domain combos with the highest number of negative values
- the **HomePage-Common** combo has an exceptionally high frequency of non-positive values and deserves special attention

|      Server       |    Domain     |*N*<br>Negative|*N*<br>Positive|*N*<br>Zero|
|-------------------|---------------|--------------:|--------------:|----------:|
|**HomePage**       |**Common**     |          13470|           8709|       4757|
|**FunctionalSetup**|**Common**     |           4604|           4605|          1|
|**Payable**        |**Financial**  |           1408|           1401|          8|
|**CoreSetup**      |**HCM**        |            944|            942|          0|
|**FinancialCommon**|**Financial**  |            644|            639|          0|
|**CoreProcesses**  |**HCM**        |            585|            578|          7|
|**CRMCommon**      |**CRM**        |            325|            320|          5|
|**GeneralLedger**  |**Financial**  |            253|            284|          2|
|**Procurement**    |**Procurement**|             46|             46|          0|
|**SCMCommon**      |**SCM**        |             45|             45|          0|

In [26]:
ct = pd.crosstab(df['Click Time Sign'], [df.Server, df.Domain]).transpose().sort_values(by='Negative', ascending=False).head(10);
ct.columns = ['*N*<br>Negative', '*N*<br>Positive', '*N*<br>Zero']
df2md(ct, ncol2bold=2)

|      Server       |    Domain     |*N*<br>Negative|*N*<br>Positive|*N*<br>Zero|
|-------------------|---------------|--------------:|--------------:|----------:|
|**HomePage**       |**Common**     |          13470|           8709|       4757|
|**FunctionalSetup**|**Common**     |           4604|           4605|          1|
|**Payable**        |**Financial**  |           1408|           1401|          8|
|**CoreSetup**      |**HCM**        |            944|            942|          0|
|**FinancialCommon**|**Financial**  |            644|            639|          0|
|**CoreProcesses**  |**HCM**        |            585|            578|          7|
|**CRMCommon**      |**CRM**        |            325|            320|          5|
|**GeneralLedger**  |**Financial**  |            253|            284|          2|
|**Procurement**    |**Procurement**|             46|             46|          0|
|**SCMCommon**      |**SCM**        |             45|             45|          0|



## Which combos have the highest % percentage of *Big Clicks*? 

- table shows the top 10 combos logging the highest percentage of clicks greater than or equal to 4,000, 8,000, or 12,000

|        Server        |    Domain     |4000+ Clicks|8000+ Clicks|12000+ Clicks|
|----------------------|---------------|-----------:|-----------:|------------:|
|**ContractManagement**|**CRM**        |       28.57|       14.29|         0.00|
|**HelpPortal**        |**Common**     |       16.67|       12.50|        12.50|
|**CoreProcesses**     |**HCM**        |       12.80|        8.30|         5.88|
|**GeneralLedger**     |**Financial**  |       10.92|        2.46|         2.11|
|**Procurement**       |**Procurement**|       10.87|        6.52|         2.17|
|**FunctionalSetup**   |**Common**     |        9.34|        3.95|         2.74|
|**Payable**           |**Financial**  |        9.14|        3.71|         2.43|
|**SCMCommon**         |**SCM**        |        8.89|        6.67|         4.44|
|**FinancialCommon**   |**Financial**  |        7.36|        3.76|         2.19|
|**CoreSetup**         |**HCM**        |        6.90|        3.29|         2.34|